In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
from numba import jit

In [18]:
N = 10000
np.random.seed(111022544)
pos = np.random.randn(N,3)
np.random.seed(111022544)
vel = np.random.randn(N,3)



@jit(nopython=True)
def G_Force(m, pos, G=1, r_soft=0.01):
    '''
    Xi,Xj = np.meshgrid(pos[:,0], pos[:,0], indexing='ij')
    Yi,Yj = np.meshgrid(pos[:,1], pos[:,1], indexing='ij')
    Zi,Zj = np.meshgrid(pos[:,2], pos[:,2], indexing='ij')
    '''
    Xi  = np.outer(np.ones((N,)),pos[:,0])
    Yi  = np.outer(np.ones((N,)),pos[:,1])
    Zi  = np.outer(np.ones((N,)),pos[:,2])
    
    Xij = Xi.T - Xi
    Yij = Yi.T - Yi
    Zij = Zi.T - Zi
    
    R = np.sqrt(Xij**2+Yij**2+Zij**2) + r_soft
    
    F = G*m**2/R**3
    
    Fx = F*Xij
    Fy = F*Yij
    Fz = F*Zij
    
    Fx_total = np.sum(Fx, axis=1)
    Fy_total = np.sum(Fy, axis=1)
    Fz_total = np.sum(Fz, axis=1)
    
    return Fx_total, Fy_total, Fz_total

#pos = np.array([[0,0,0],[1,1,1]])
t0 = time.time()
Fx_total, Fy_total, Fz_total = G_Force(1, pos)
t1 = time.time()
print(Fx_total)
print(Fy_total)
print(Fz_total)
print(t1-t0)

[  493.68275866    51.11255907  -202.16665181 ...  -643.94725799
 -1198.52751254  -440.85683443]
[1460.05715724  467.09827586 1441.25178345 ... -137.87849003  792.22251754
 1665.49387317]
[ -961.8335539    947.33482936  1367.113171   ... -1057.36124058
  -631.08404045  1051.9727533 ]
11.796590805053711


In [4]:
pos = np.array([[0],[1],[2],[3]])
Xi  = np.outer(np.ones((4,)),pos[:,0])
print(Xi)

xi,xj = np.meshgrid(pos[:,0], pos[:,0], indexing='xy')
print(Xi.T-Xi)

[[0. 1. 2. 3.]
 [0. 1. 2. 3.]
 [0. 1. 2. 3.]
 [0. 1. 2. 3.]]
[[ 0. -1. -2. -3.]
 [ 1.  0. -1. -2.]
 [ 2.  1.  0. -1.]
 [ 3.  2.  1.  0.]]


In [24]:
1/(np.sqrt(3)+0.01)**3

0.18915487922589125

In [21]:
1/2.01**3*2

0.2462871898274525

In [17]:
37.166444063186646 #N=10000 no jit
12.239506006240845 #N=10000 with jit

12.239506006240845